In [ ]:
import sys
import os
import subprocess
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import cProfile
# from line_profiler import LineProfiler

# Add the path to housing_app to the Python path, if it's not in the folder
# TODO: Adjust the path to match the local location of housing_app
# sys.path.append('/Users/k4robins/Devel/0_Thesis/housing_app')
# from housing_app.model.model import City

# Alternatively, if the housing_app is in the current folder, directly import it
from model.model import City

## Single Run

In [ ]:
# Set parameter values
num_steps  = 40
parameters = {
            'run_notes': 'Debugging model.',
            'width': 10,
            'height': 10,
            'init_city_extent': 10.,  # f CUT OR CHANGE?
            'seed_population': 1,
            'density': 1,
            'subsistence_wage': 40000.,  # psi
            'init_wage_premium_ratio': 0.2,
            'workforce_rural_firm': 100,
            'price_of_output': 1.,  # TODO CUT?
            'alpha_F': 0.18,
            'beta_F': 0.72,  # beta and was lambda, workers_share of aglom surplus
            'beta_city': 1.12,
            'gamma': 0.02,  # FIX value
            'Z': 0.5,  # CUT? Scales new entrants
            'firm_adjustment_parameter': 0.25,
            'wage_adjustment_parameter': 0.5,
            'mortgage_period': 5.0,  # T, in years
            'working_periods': 4000,  # in years
            'savings_rate': 0.3,
            'r_prime': 0.05,  # 0.03
            'discount_rate': 0.07, # 1/delta
            'r_margin': 0.01,
            'property_tax_rate': 0.04,  # tau, annual rate, was c
            'housing_services_share': 0.3,  # a
            'maintenance_share': 0.2,  # b
            'max_mortgage_share': 0.9,
            'ability_to_carry_mortgage': 0.28,
            'wealth_sensitivity': 0.1,
        }

def run_simulation(num_steps, parameters):
    city = City(num_steps, **parameters)
    city.run_model()

    agent_out = city.datacollector.get_agent_vars_dataframe()
    model_out = city.datacollector.get_model_vars_dataframe()
    return agent_out, model_out


agent_out, model_out = run_simulation(num_steps, parameters)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Filter agent_out data for Land agents
df = agent_out.query("agent_type == 'Land'")
df = df.dropna(axis=1, how='all').reset_index(drop=True)
df = df.reset_index(drop=True)

# Get the range of time steps
time_steps = df['time_step'].unique()
middle_time_step = time_steps[len(time_steps) // 2]

# Calculate min and max dot sizes
min_dot_size = 10
max_dot_size = 70

# Create a scatter plot with three dots
fig, ax = plt.subplots(figsize=(8, 6))

# Create dots for the three selected time steps
for i, time_step in enumerate([time_steps[0], middle_time_step, time_steps[-1]]):
    land_agents = df.query("time_step == @time_step")
    
    # Calculate dot size based on time step
    normalized_dot_size = (time_step - min(time_steps)) / (max(time_steps) - min(time_steps))
    dot_size = min_dot_size + (max_dot_size - min_dot_size) * normalized_dot_size
    
    # Create scatter plot for distance from center vs warranted price
    ax.scatter(land_agents['distance_from_center'], land_agents['warranted_price'], label=f'Time Step {time_step}', alpha=0.7, s=dot_size)

# Set labels and title
ax.set_xlabel('Distance from Center')
ax.set_ylabel('Warranted Price')
ax.set_title('Distance vs Warranted Price')
ax.legend()

plt.show()


In [ ]:
def plot_model_data(model_out):
    workers = np.array(model_out['workers'])
    wage_premium = np.array(model_out['wage_premium'])
    wage    = np.array(model_out['wage'])
    city_extent_calc = np.array(model_out['city_extent_calc'])
    p_dot   = np.array(model_out['p_dot'])
    time    = np.arange(len(workers))

    # Set up the figure with subplots
    fig, axes = plt.subplots(4, 2, figsize=(10, 15))
    fig.suptitle('Model Output', fontsize=16)

    # Subplot 0L: Evolution of the Wage  
    axes[0, 0].plot(time, wage, color='red')
    axes[0, 0].set_title('Evolution of the Wage (Rises)')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('Wage')
    axes[0, 0].grid(True)

    # Subplot 0R: Evolution of the City Extent
    axes[0,1].plot(time, city_extent_calc, color='red')
    axes[0,1].set_title('Evolution of the City Extent (Rises)')
    axes[0,1].set_xlabel('Time')
    axes[0,1].set_ylabel('City Extent')
    axes[0,1].grid(True)

    # Subplot 1L: Evolution of the Workforce
    axes[1, 0].plot(time, workers, color='purple') 
    axes[1, 0].set_title('Evolution of the Workforce (Rises)')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Workers')
    axes[1, 0].grid(True)

    # Subplot 1R: City Extent and Workforce  
    axes[1, 1].plot(city_extent_calc, workers, color='magenta')
    axes[1, 1].set_title('City Extent and Workforce (Curves Up)')
    axes[1, 1].set_xlabel('City Extent')
    axes[1, 1].set_ylabel('Workers')
    axes[1, 1].grid(True)              
    
    # Subplot 2L: City Extent and Wage
    axes[2, 0].plot(time, city_extent_calc, color='red')
    axes[2, 0].set_title('City Extent and Wage (Curves Up)')
    axes[2, 0].set_xlabel('Wage')
    axes[2, 0].set_ylabel('City Extent')
    axes[2, 0].grid(True)

    # Subplot 2R: Workforce Response to Wage
    axes[2, 1].plot(wage, workers, color='purple')
    axes[2, 1].set_title('Workforce Response to Wage')
    axes[2, 1].set_xlabel('Wage')
    axes[2, 1].set_ylabel('Workers')
    axes[2, 1].grid(True)

    # Subplot 3L: Evolution of P_Dot
    axes[3, 0].plot(time, time, color='green')
    axes[3, 0].set_title('Evolution of P-Dot')
    axes[3, 0].set_xlabel('Time')
    axes[3, 0].set_ylabel('P-Dot')
    axes[3, 0].grid(True)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()
    
plot_model_data(model_out)



### Additional Plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Filter agent_out data for Land agents
df = agent_out.query("agent_type == 'Land'")
df = df.dropna(axis=1, how='all').reset_index(drop=True)
df = df.reset_index(drop=True)

# Get the range of time steps
time_steps = df['time_step'].unique()
num_time_steps = len(time_steps)
middle_time_step = time_steps[num_time_steps // 2]

# Find the overall min and max values for the color scale
min_price = df['warranted_price'].min()
max_price = df['warranted_price'].max()

# Create subplots with 2 rows and 3 columns
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Create a common colorbar axis
divider = make_axes_locatable(axs[0, -1])
cax = divider.append_axes("right", size="5%", pad=0.1)

# Iterate over the subplots and time steps
for i, time_step in enumerate([time_steps[0], middle_time_step, time_steps[-1]]):
    land_agents = df.query("time_step == @time_step")
    
    # Create a grid to represent the space
    grid_size = (df['x'].max() + 1, df['y'].max() + 1)
    heatmap = np.zeros(grid_size)
    
    # Fill the heatmap with the values
    for index, row in land_agents.iterrows():
        x = int(row['x'])
        y = int(row['y'])
        price = row['warranted_price']
        heatmap[x, y] = price
    
    # Display the heatmap in the current subplot
    im = axs[0, i].imshow(heatmap, cmap='viridis', origin='lower', extent=[0, grid_size[0], 0, grid_size[1]], vmin=min_price, vmax=max_price)
    axs[0, i].set_title(f'Time Step {time_step}')
    axs[0, i].set_xlabel('X')
    axs[0, i].set_ylabel('Y')
    axs[0, i].grid(False)
    
    # Create scatter plot for distance from center vs warranted price
    scatter = axs[1, i].scatter(land_agents['distance_from_center'], land_agents['warranted_price'], c='blue', alpha=0.5)
    axs[1, i].set_title(f'Distance vs Warranted Price at Time Step {time_step}')
    axs[1, i].set_xlabel('Distance from Center')
    axs[1, i].set_ylabel('Warranted Price')
    axs[1, i].grid(True)
    
    # Set same min and max values for scatter plots
    axs[1, i].set_ylim(min_price, max_price)

# Add a common colorbar
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Warranted Price')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Filter agent_out data for Land agents
df = agent_out.query("agent_type == 'Land'")
df = df.dropna(axis=1, how='all').reset_index(drop=True)
df = df.reset_index(drop=True)

# Get the range of time steps
time_steps = df['time_step'].unique()
num_time_steps = len(time_steps)
middle_time_step = time_steps[num_time_steps // 2]

# Find the overall min and max values for the color scale
min_price = df['warranted_price'].min()
max_price = df['warranted_price'].max()

# Create subplots with 1 row and 3 columns
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Create a common colorbar axis
divider = make_axes_locatable(axs[-1])
cax = divider.append_axes("right", size="5%", pad=0.1)

# Iterate over the subplots and time steps
for i, time_step in enumerate([time_steps[0], middle_time_step, time_steps[-1]]):
    land_agents = df.query("time_step == @time_step")
    
    # Create a grid to represent the space
    grid_size = (df['x'].max() + 1, df['y'].max() + 1)
    heatmap = np.zeros(grid_size)
    
    # Fill the heatmap with the values
    for index, row in land_agents.iterrows():
        x = int(row['x'])
        y = int(row['y'])
        price = row['warranted_price']
        heatmap[x, y] = price
    
    # Display the heatmap in the current subplot
    im = axs[i].imshow(heatmap, cmap='viridis', origin='lower', extent=[0, grid_size[0], 0, grid_size[1]], vmin=min_price, vmax=max_price)
    axs[i].set_title(f'Time Step {time_step}')
    axs[i].set_xlabel('X')
    axs[i].set_ylabel('Y')
    axs[i].grid(False)

# Add a common colorbar
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Warranted Price')

plt.tight_layout()
plt.show()


### Model execution alternatives

In [ ]:
# # Run with profiling
# # Function profiling
# def run_simulation(num_steps, parameters):
#     city = City(num_steps, **parameters)

#     profiler = cProfile.Profile()
#     profiler.enable()

#     city.run_model()

#     profiler.disable()
#     profiler.print_stats(sort='cumulative')
    
#     agent_out = city.datacollector.get_agent_vars_dataframe()
#     model_out = city.datacollector.get_model_vars_dataframe()
#     return agent_out, model_out

# # Line profiling
# def run_simulation(num_steps, parameters):
#     city = City(num_steps, **parameters)
#     profiler = LineProfiler()

# #     profiler.add_function(city.run_model)
# #     profiler.add_function(city.datacollector.get_agent_vars_dataframe)
# #     profiler.add_function(city.datacollector.get_model_vars_dataframe)
#     profiler.add_function(city.person.run)

#     profiler.enable()

#     city.run_model()

#     profiler.disable()
#     profiler.print_stats()

#     agent_out = city.datacollector.get_agent_vars_dataframe()
#     model_out = city.datacollector.get_model_vars_dataframe()
#     return agent_out, model_out

# Call the functions to generate and display the plots

In [ ]:
# # Bath execution
# from mesa.batchrunner import batch_run

# variable_parameters = {
#     'density': [1, 100],
#     'subsistence_wage': [10000, 30000]
# }

# agent_reporters = {
#     'x': lambda a: a.pos[0],
#     'y': lambda a: a.pos[1],
# }

# # Run the batch simulations
# results = batch_run(City, variable_parameters, data_collection_period=1, iterations=2, max_steps=4)



In [ ]:
# # Convert batch results to a pandas DataFrame
# df = pd.DataFrame(results)

# # Save the DataFrame to a CSV file
# df.to_csv('batch_results.csv', index=False)